# 0. Prepara o spark e java, encontra o spark, cria contexto
tente procurar instruções como instalar o SW, dependendo do sistema operativo. De seguida, no Mac OS

In [2]:
import os;


os.environ["SPARK_HOME"] = "/Users/pedro/servers/spark/spark-3.2.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] ="/Library/Java/JavaVirtualMachines/adoptopenjdk-15.jdk/Contents/Home"

#os.environ["SPARK_HOME"] = "/Users/pedro/servers/spark-3.1.1-bin-hadoop2.7"
#os.environ["JAVA_HOME"]="/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home"
!java -version

openjdk version "15.0.2" 2021-01-19
OpenJDK Runtime Environment AdoptOpenJDK (build 15.0.2+7)
OpenJDK 64-Bit Server VM AdoptOpenJDK (build 15.0.2+7, mixed mode, sharing)


# Prepara o pyspark e spark context

In [3]:
import findspark 

findspark.init()

import pyspark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
print ("Running Spark Version ", spark.version)

#OLD deprecated:
#from pyspark import SparkContext

#sc = SparkContext('local') 
#print ("Running Spark Version ", sc.version)

Running Spark Version  3.2.1


# Tive de resolver um erro, a correr no Mac OS sem rede:

# Continuacao

In [ ]:
from pyspark.conf import SparkConf
conf = SparkConf()
print (conf.toDebugString())

# PART 1- My first pyspark app

# What does the next code do? 

# what changes if you increase the number of samples?
# why?

In [ ]:
# useful to have this code snippet to avoid getting an error in case forgeting
# to close spark

try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

# import python libraries
import random

# initialize
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
num_samples = 10
#num_samples = 100000000

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = spark.sparkContext.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)

# PART 1- SQL

# Now lets also do some basic SQL with tables emp and dep

In [ ]:
employees = spark.read.json('/Users/pedro/Documents/Aulas/SGD/2022SGD/pratica/aula5_pySpark/employee.json')

# Print the schema in a tree format

employees.printSchema()

employees.select("name").show(20)

In [ ]:
dep = spark.read.json('/Users/pedro/Documents/Aulas/SGD/2020SGD/praticas/lab6_SparkSparkSQL/dep.json')
# Print the schema in a tree format

dep.printSchema()

dep.select("*").show(20)

In [ ]:
dep.createOrReplaceTempView("dep")
employees.createOrReplaceTempView("employees")

# write a query to select all employees aged > 23

In [ ]:
spark.sql("select * from employees where age>23").show()

# write a query to show number of employees for each age

In [ ]:
ageGroups = spark.sql("?")
ageGroups.show()

# Show all info of departments of each employee together with all the employee info

In [ ]:
employees.registerTempTable("emp")
empdep = spark.sql("?")
empdep.show()

# Same, but restrict to name of employee and of dep

In [ ]:
empdep = spark.sql("?")
empdep.show()

# Finally, show employee and department name but only for department SALES

In [ ]:
empVendas = spark.sql("?")
empVendas.show()

# PART 2 - Working with data and statistics in pyspark

# read cars dataset, show nr of rows, col names and first row

In [5]:
from pyspark import SparkContext


sc = SparkContext.getOrCreate();

In [6]:
import os
#default_path="/Users/pedro/Documents/Aulas/SGD/2019SGD/praticas/empreparacao/car-data"
#os.chdir(default_path)
print("Current Working Directory " , os.getcwd())

inp_file = sc.textFile("car-data/car-milage.csv")
car_rddText = inp_file.map(lambda line: line.split(','))

Current Working Directory  /Users/pedro/Documents/Aulas/SGD/2022SGD/pratica/aula5_pySpark


In [7]:
car_rddText.count()

33

In [8]:
titles=car_rddText.first()
print(titles)

['mpg', 'displacement', 'hp', 'torque', 'CRatio', 'RARatio', 'CarbBarrells', 'NoOfSpeed', 'length', 'width', 'weight', 'automatic']


In [ ]:
inp_file = sc.textFile("car-data/car-milage-no-hdr.csv")
car_rdd = inp_file.map(lambda line: line.split(','))

In [ ]:
print(car_rdd.first())

# what are we doing differently in the next code?

In [ ]:
from array import array

inp_file = sc.textFile("car-data/car-milage-no-hdr.csv")
car_rdd = inp_file.map(lambda line: [float(x) for x in line.split(',')])
car_rdd.first()

# mean miles per galon?
# Max & min Horse Power?
# Variance of cars weights?

In [ ]:
from pyspark.mllib.stat import Statistics
summary = Statistics.colStats(car_rdd)
summary.count()
print(*titles)
print('mean/min/max/var')
summary.mean(),summary.min(),summary.max(), summary.variance()

# What do the following correlations say about the attrs? What about the negative values? 
hp com weight?
ra_ratio com width?

In [ ]:
hp = car_rdd.map(lambda x: x[2])
weight = car_rdd.map(lambda x: x[10])
print ('%2.3f' % Statistics.corr(hp, weight, method="pearson") )
print ('%2.3f' % Statistics.corr(hp, weight, method="spearman") )

In [ ]:
ra_ratio = car_rdd.map(lambda x: x[5])
width = car_rdd.map(lambda x: x[9])
print ('%2.3f' % Statistics.corr(ra_ratio, width, method="pearson") )
print ('%2.3f' % Statistics.corr(ra_ratio, width, method="spearman") )

# Recreate the data as a dataframe, show the contents, drop na

In [ ]:
import pandas as pd

cars = pd.read_csv("car-data/car-milage.csv")

In [ ]:
cars

In [ ]:
cars=cars.dropna()

# Use describe for statistics. Is there a big or small var in miles per galon?

In [ ]:
cars.describe()

# To continue ....